<a href="https://colab.research.google.com/github/maidacundo/test_referees_optimization/blob/main/test_ortools3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install ortools
from ortools.sat.python import cp_model
import multiprocessing
import random
import numpy as np

In [11]:
def make_day(num_teams, day):
    # using circle algorithm, https://en.wikipedia.org/wiki/Round-robin_tournament#Scheduling_algorithm
    assert not num_teams % 2, "Number of teams must be even!"
    # generate list of teams
    lst = list(range(num_teams))
    # rotate
    day %= (num_teams - 1)  # clip to 0 .. num_teams - 2
    if day:                 # if day == 0, no rotation is needed (and using -0 as list index will cause problems)
        lst = lst[:1] + lst[-day:] + lst[1:-day]
    # pair off - zip the first half against the second half reversed
    half = num_teams // 2
    return list(zip(lst[:half], lst[half:][::-1]))

def make_schedule(num_teams):
    """
    Produce a double round-robin schedule
    """
    # number of teams must be even
    if num_teams % 2:
        num_teams += 1  # add a dummy team for padding

    # build first round-robin
    schedule = [make_day(num_teams, day) for day in range(num_teams - 1)]
    # generate second round-robin by swapping home,away teams
    swapped = [[(away, home) for home, away in day] for day in schedule]

    return schedule + swapped

In [12]:
def make_prediction(num_referee, num_teams, calendar, referee_history, seconds_number=30, print_bool = False, min_referee_per_match=3):

  class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
      """Print intermediate solutions."""

      def __init__(self, variables):
          cp_model.CpSolverSolutionCallback.__init__(self)
          self.__variables = variables
          self.__solution_count = 0

      def on_solution_callback(self):
          self.__solution_count += 1
          print(f'trovata soluzione num {self.__solution_count}')
          print(f'Differenza totale per partite arbitrate: {self.Value(total_diff_per_referre_to_minimize)}')
          print(f'Differenza totale per squadre arbitrate: {self.Value(team_diff_per_referre_to_minimize)}')

      def solution_count(self):
          return self.__solution_count

  model = cp_model.CpModel()

  x = []
  for i in range(num_referee):
      r = []
      for day in range(num_days):
          num_matches = len(calendar[day])
          d = []
          for j in range(num_matches):
              d.append(model.NewIntVar(0, 1, 'referee{}_day{}_match{}'.format(i, day, j)))
          r.append(d)
      x.append(r)

  # Ogni arbitro è assegnato al max ad una partita per giorno
  for i in range(num_referee):
      for d in range(num_days):
          model.AddAtMostOne(x[i][d][j] for j in range(num_matches))

  # Ogni partita ha 3 arbitri
  for d in range(num_days):
      for j in range(num_matches):
          model.Add(sum([x[i][d][j] for i in range(num_referee)]) == min_referee_per_match)

  # calcolo delle squadre arbitrate da ogni singolo arbitro
  referee_matches_per_team = []
  for i in range(num_referee):
    teams_per_referee = []
    for s in range(num_teams):
      matches_per_team = []
      for d in range(num_days):
        for m in calendar[d]:
          if s in m:
            j = calendar[d].index(m)
            matches_per_team.append(x[i][d][j])
      try:
        previous_matches_number = referee_history[i][s]
      except IndexError:
        print('errore')
        previous_matches_number = 0
      teams_per_referee.append(sum(matches_per_team) + previous_matches_number)
    referee_matches_per_team.append(teams_per_referee)

  team_diff = []
  all_variables = []

  for i in range(len(referee_matches_per_team)):
    t = []
    for s in range(num_teams):
      c = []
      for j in range(len(referee_matches_per_team)):
        if i != j:
          diff_var = model.NewIntVar(0, 100, f'matches_diff_{i}_{j}_team_{s}') # todo controllare meglio up e lb!
          abs_diff = model.AddAbsEquality(diff_var, (referee_matches_per_team[i][s] - referee_matches_per_team[j][s]))
          c.append(diff_var)
          all_variables.append(diff_var)
      
      t.append(sum(c))
    team_diff.append(sum(t))

  # aggiungo una decision stategy (euristica) per migliorare le performance
  model.AddDecisionStrategy(all_variables, cp_model.CHOOSE_LOWEST_MIN, cp_model.SELECT_MIN_VALUE)

  team_diff_per_referre_to_minimize = sum(team_diff)

  # calcolo dei match totali arbitrati da ogni singolo arbitro
  total_matches_per_referee = []

  for i in range(num_referee):
      total_matches = 0
      for d in range(num_days):
          total_matches += sum(x[i][d])
      total_matches_per_referee.append(total_matches)

  matches_differences = []
  all_variables = []

  for i in range(len(total_matches_per_referee)):
      c = []
      for j in range(len(total_matches_per_referee)):
          if i != j:
              diff_var = model.NewIntVar(0, 100, f'matches_diff_{i}_{j}')
              abs_diff = model.AddAbsEquality(diff_var, (total_matches_per_referee[i] - total_matches_per_referee[j]))
              c.append(diff_var)
              all_variables.append(diff_var)
      matches_differences.append(c)

  # aggiungo una decision stategy (euristica) per migliorare le performance
  model.AddDecisionStrategy(all_variables, cp_model.CHOOSE_LOWEST_MIN, cp_model.SELECT_LOWER_HALF)

  total_differences_per_referee = []
  for i in range(len(matches_differences)):
      total_differences_per_referee.append(sum(matches_differences[i]))
    
  total_diff_per_referre_to_minimize = sum(total_differences_per_referee)

  # lista contenente tutti i valori da minimizzare
  values_to_minimize = []
  values_to_minimize.append(team_diff_per_referre_to_minimize * 0.7)
  values_to_minimize.append(total_diff_per_referre_to_minimize * 0.3)

  model.Minimize(sum(values_to_minimize))
  # model.Minimize(team_diff_per_referre_to_minimize)

  if print_bool:
    print(model.ModelStats())

  solver = cp_model.CpSolver()
  solution_printer = VarArraySolutionPrinter(x)

  #solver.parameters.enumerate_all_solutions = True
  solver.parameters.num_search_workers = multiprocessing.cpu_count()

  solver.parameters.max_time_in_seconds = seconds_number

  status = solver.Solve(model, solution_printer)

  solution = []

  if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:

      if print_bool:
        print('POSSIBILE SOLUZIONE')
        
      match_per_referee = [0] * num_referee

      for i in range(num_referee):
          solution.append([])
          num_matches = len(calendar[d])
          for d in range(num_days):
              for j in range(num_matches):
                  if solver.BooleanValue(x[i][d][j]):
                      match_per_referee[i] += 1
                      if print_bool:
                        print(
                            f'Referee {i} assigned to match {calendar[d][j]}')
                      solution[i] = calendar[d][j]
      print()
      
      if print_bool:
        print(f'Differenza totale per partite arbitrate: {solver.Value(total_diff_per_referre_to_minimize)}')
        print(f'Differenza totale per squadre arbitrate: {solver.Value(team_diff_per_referre_to_minimize)}')

      print()
      scores_team_diff = []
      if print_bool:
        print('TOTALI PARTITE PER SQUADRA')
        for i in range(num_referee):
          print(f'Referee {i}: {solver.Value(team_diff[i])}')
          scores_team_diff.append(solver.Value(team_diff[i]))

  return solution

In [13]:
def split_list(alist, wanted_parts):
    # random.shuffle(alist)
    length = len(alist)
    return [ alist[i*length // wanted_parts: (i+1)*length // wanted_parts] 
             for i in range(wanted_parts) ]

In [14]:
def sort_referees(referees_list, giornata_split, referee_history):
  squadre_giornata = []
  for d in range(len(giornata_split)):
    for m in giornata_split[d]:
      for t in m:
        squadre_giornata.append(t)

  scoring_referee_dict = {}

  for i in referees_list:
    sum = 0
    for j in squadre_giornata:
      sum += referee_history[i][j]
    scoring_referee_dict[i] = sum

  sorted_list = sorted(scoring_referee_dict.items(), key=lambda x: x[1])
  sorted_referees = []
  print(sorted_list)
  for e in sorted_list:
    sorted_referees.append(e[0])

  return sorted_referees

In [15]:
def remove_referees_from_list(referees_list, referre_to_remove):
  for el in referre_to_remove:
    referees_list.remove(el)
  return referees_list

In [16]:
def split_data_to_dict(splitting_number, referees_list, giornata):
  giornata_split_list = split_list(giornata[0], splitting_number)
  splitting_dict = {}
  for i in range(splitting_number):
    # partite vengono splittate
    giornata_split_list[i] = [giornata_split_list[i]]

    # arbitri vengono sortati per la quantità di partite arbitrate per le squadre della giornata
    sorted_referees = sort_referees(referees_list, giornata_split_list[i], referee_history)

    # split dei referee 
    referee_split_list = split_list(sorted_referees, splitting_number - i)

    # rimuovo gli arbitri che hanno già arbitrato
    referees_list = remove_referees_from_list(referees_list, referee_split_list[0])
    splitting_dict[i] = (giornata_split_list[i], referee_split_list[0])

  return splitting_dict

In [28]:
num_referee = 40
num_teams = 16
splitting_number = 1

calendar = make_schedule(num_teams)

num_days = len(calendar)
min_referee_per_match = 3

referee_history = []
for i in range(num_referee):
  referee_history.append([])
  for s in range(num_teams):
    referee_history[i].append(0)

referees_list = []
for i in range(num_referee):
  referees_list.append(i)

In [18]:
for i in range(len(calendar)):
  giornata = calendar[i:i+1]
  print(f'Varianza history: {np.var(referee_history)}')

  # split dei match e degli arbitri in maniera pseudo randomica (non randomica)
  splitting_dict = split_data_to_dict(splitting_number, referees_list.copy(), giornata)

  for split_n in range(splitting_number):
    giornata_split = splitting_dict[split_n][0]
    referee_split = splitting_dict[split_n][1]

    # remapping della history
    referee_history_split = []
    for r in referee_split:
      referee_history_split.append(referee_history[r])
      
    print(giornata_split)
    num_days = len(giornata_split)
    partial_solution = make_prediction(len(referee_split), len(giornata_split[0]) * 2, giornata_split, referee_history_split, seconds_number=10)

    for i in range(len(referee_split)):
      for j in partial_solution[i]:
        # print(f'referee {referee_split[i]} assigned to {j}')
        referee_history[referee_split[i]][j] += 1

Varianza history: 0.0
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (15, 0), (16, 0), (17, 0), (18, 0), (19, 0), (20, 0), (21, 0), (22, 0), (23, 0), (24, 0), (25, 0), (26, 0), (27, 0), (28, 0), (29, 0), (30, 0), (31, 0), (32, 0), (33, 0), (34, 0), (35, 0), (36, 0), (37, 0), (38, 0), (39, 0)]
[[(0, 15), (1, 14), (2, 13), (3, 12), (4, 11), (5, 10), (6, 9), (7, 8)]]
trovata soluzione num 1
Differenza totale per partite arbitrate: 768
Differenza totale per squadre arbitrate: 3552


Varianza history: 0.069375
[(24, 0), (25, 0), (26, 0), (27, 0), (28, 0), (29, 0), (30, 0), (31, 0), (32, 0), (33, 0), (34, 0), (35, 0), (36, 0), (37, 0), (38, 0), (39, 0), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 2), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (15, 2), (16, 2), (17, 2), (18, 2), (19, 2), (20, 2), (21, 2), (22, 2), (23, 2)]
[[(0, 14), (15, 13), (1, 12), (2, 11), (3, 10), (4, 9), (5,

IndexError: ignored

In [ ]:
referee_history

In [ ]:
# codice vecchio
for i in range(len(calendar)):

  print(f'Varianza history: {np.var(referee_history)}')
  giornata = calendar[i:i+1]

  # shuffle & splitting delle liste
  giornata_split_list = split_list(giornata[0], splitting_number)
  for i in range(splitting_number):
    giornata_split_list[i] = [giornata_split_list[i]]
  # shuffle & splitting degli arbitri
  referee_split_list = split_list(referees_list, splitting_number)

  for i in range(splitting_number):
    print(f'Splitting n. {i}')
    referee_split = referee_split_list[i]
    giornata_split = giornata_split_list[i]

    # remapping della history
    referee_history_split = []
    for r in referee_split:
      referee_history_split.append(referee_history[r])
    print(giornata_split)
    num_days = len(giornata_split)
    # partial_solution = make_prediction(len(referee_split), len(giornata_split[0]) * 2, giornata_split, referee_history_split)

    for i in range(len(referee_split)):
      for j in partial_solution[i]:
        # print(f'referee {referee_split[i]} assigned to {j}')
        referee_history[referee_split[i]][j] += 1

In [ ]:
# codice vecchio

for i in range(len(calendar)):
  giornata = calendar[i:i+1]
  print(giornata)
  num_days = len(giornata)
  solution = make_prediction(num_referee, num_teams, giornata, referee_history)
  for i in range(num_referee):
    for j in solution[i]:
      referee_history[i][j] += 1

[[(0, 15), (1, 14), (2, 13), (3, 12), (4, 11), (5, 10), (6, 9), (7, 8)]]
trovata soluzione num 1
Differenza totale per partite arbitrate: 768
Differenza totale per squadre arbitrate: 3552


[[(0, 14), (15, 13), (1, 12), (2, 11), (3, 10), (4, 9), (5, 8), (6, 7)]]
trovata soluzione num 1
Differenza totale per partite arbitrate: 768
Differenza totale per squadre arbitrate: 6780
trovata soluzione num 2
Differenza totale per partite arbitrate: 768
Differenza totale per squadre arbitrate: 6548


[[(0, 13), (14, 12), (15, 11), (1, 10), (2, 9), (3, 8), (4, 7), (5, 6)]]
trovata soluzione num 1
Differenza totale per partite arbitrate: 768
Differenza totale per squadre arbitrate: 9608
trovata soluzione num 2
Differenza totale per partite arbitrate: 768
Differenza totale per squadre arbitrate: 9576
trovata soluzione num 3
Differenza totale per partite arbitrate: 768
Differenza totale per squadre arbitrate: 9472


[[(0, 12), (13, 11), (14, 10), (15, 9), (1, 8), (2, 7), (3, 6), (4, 5)]]
trovata solu

In [26]:
num_days = len(calendar)
solution = make_prediction(num_referee, num_teams, calendar, referee_history, seconds_number=480)
for i in range(num_referee):
  for j in solution[i]:
    referee_history[i][j] += 1

IndexError: ignored

In [ ]:
# test con gli hints

def make_prediction_with_hints(num_referee, num_teams, calendar, referee_history, seconds_number=30, print_bool = False, min_referee_per_match=3):

  class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
      """Print intermediate solutions."""

      def __init__(self, variables):
          cp_model.CpSolverSolutionCallback.__init__(self)
          self.__variables = variables
          self.__solution_count = 0

      def on_solution_callback(self):
          self.__solution_count += 1
          print(f'trovata soluzione num {self.__solution_count}')
          print(f'Differenza totale per partite arbitrate: {self.Value(total_diff_per_referre_to_minimize)}')
          print(f'Differenza totale per squadre arbitrate: {self.Value(team_diff_per_referre_to_minimize)}')

      def solution_count(self):
          return self.__solution_count

  model = cp_model.CpModel()

  x = []
  for i in range(num_referee):
      r = []
      for day in range(num_days):
          num_matches = len(calendar[day])
          d = []
          for j in range(num_matches):
              d.append(model.NewIntVar(0, 1, 'referee{}_day{}_match{}'.format(i, day, j)))
          r.append(d)
      x.append(r)

  # Ogni arbitro è assegnato al max ad una partita per giorno
  for i in range(num_referee):
      for d in range(num_days):
          model.AddAtMostOne(x[i][d][j] for j in range(num_matches))

  # Ogni partita ha 3 arbitri
  for d in range(num_days):
      for j in range(num_matches):
          model.Add(sum([x[i][d][j] for i in range(num_referee)]) == min_referee_per_match)

  # calcolo delle squadre arbitrate da ogni singolo arbitro
  referee_matches_per_team = []
  for i in range(num_referee):
    teams_per_referee = []
    for s in range(num_teams):
      matches_per_team = []
      for d in range(num_days):
        for m in calendar[d]:
          if s in m:
            j = calendar[d].index(m)
            matches_per_team.append(x[i][d][j])
      try:
        previous_matches_number = referee_history[i][s]
      except IndexError:
        print('errore')
        previous_matches_number = 0
      teams_per_referee.append(sum(matches_per_team) + previous_matches_number)
    referee_matches_per_team.append(teams_per_referee)

  team_diff = []
  all_variables = []

  for i in range(len(referee_matches_per_team)):
    t = []
    for s in range(num_teams):
      c = []
      for j in range(len(referee_matches_per_team)):
        if i != j:
          diff_var = model.NewIntVar(0, 100, f'matches_diff_{i}_{j}_team_{s}') # todo controllare meglio up e lb!
          abs_diff = model.AddAbsEquality(diff_var, (referee_matches_per_team[i][s] - referee_matches_per_team[j][s]))
          c.append(diff_var)
          all_variables.append(diff_var)
      
      t.append(sum(c))
    team_diff.append(sum(t))

  # aggiungo una decision stategy (euristica) per migliorare le performance
  model.AddDecisionStrategy(all_variables, cp_model.CHOOSE_LOWEST_MIN, cp_model.SELECT_MIN_VALUE)

  team_diff_per_referre_to_minimize = sum(team_diff)

  # calcolo dei match totali arbitrati da ogni singolo arbitro
  total_matches_per_referee = []

  for i in range(num_referee):
      total_matches = 0
      for d in range(num_days):
          total_matches += sum(x[i][d])
      total_matches_per_referee.append(total_matches)

  matches_differences = []
  all_variables = []

  for i in range(len(total_matches_per_referee)):
      c = []
      for j in range(len(total_matches_per_referee)):
          if i != j:
              diff_var = model.NewIntVar(0, 100, f'matches_diff_{i}_{j}')
              abs_diff = model.AddAbsEquality(diff_var, (total_matches_per_referee[i] - total_matches_per_referee[j]))
              c.append(diff_var)
              all_variables.append(diff_var)
      matches_differences.append(c)

  # aggiungo una decision stategy (euristica) per migliorare le performance
  model.AddDecisionStrategy(all_variables, cp_model.CHOOSE_LOWEST_MIN, cp_model.SELECT_LOWER_HALF)

  total_differences_per_referee = []
  for i in range(len(matches_differences)):
      total_differences_per_referee.append(sum(matches_differences[i]))
    
  total_diff_per_referre_to_minimize = sum(total_differences_per_referee)

  # lista contenente tutti i valori da minimizzare
  values_to_minimize = []
  values_to_minimize.append(team_diff_per_referre_to_minimize * 0.7)
  values_to_minimize.append(total_diff_per_referre_to_minimize * 0.3)

  model.Minimize(sum(values_to_minimize))
  # model.Minimize(team_diff_per_referre_to_minimize)

  if print_bool:
    print(model.ModelStats())

  solver = cp_model.CpSolver()
  solution_printer = VarArraySolutionPrinter(x)

  #solver.parameters.enumerate_all_solutions = True
  solver.parameters.num_search_workers = multiprocessing.cpu_count()

  solver.parameters.max_time_in_seconds = seconds_number

  status = solver.Solve(model, solution_printer)

  solution = []

  if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:

      if print_bool:
        print('POSSIBILE SOLUZIONE')
        
      match_per_referee = [0] * num_referee

      for i in range(num_referee):
          solution.append([])
          num_matches = len(calendar[d])
          for d in range(num_days):
              for j in range(num_matches):
                  if solver.BooleanValue(x[i][d][j]):
                      match_per_referee[i] += 1
                      if print_bool:
                        print(
                            f'Referee {i} assigned to match {calendar[d][j]}')
                      solution[i] = calendar[d][j]
      print()
      
      if print_bool:
        print(f'Differenza totale per partite arbitrate: {solver.Value(total_diff_per_referre_to_minimize)}')
        print(f'Differenza totale per squadre arbitrate: {solver.Value(team_diff_per_referre_to_minimize)}')

      print()
      scores_team_diff = []
      if print_bool:
        print('TOTALI PARTITE PER SQUADRA')
        for i in range(num_referee):
          print(f'Referee {i}: {solver.Value(team_diff[i])}')
          scores_team_diff.append(solver.Value(team_diff[i]))

  return solution

In [22]:
import numpy as np

# 0.731875
# 0.474375
# 0.380625
# 0.906875

np.var(referee_history)

0.42437500000000006